In [1]:
#Library for edit dataset
import pandas as pd
import numpy as np
import datetime as dp

#Library for visualization
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import folium 
from folium.plugins import StripePattern
import branca.colormap
from collections import defaultdict
from folium.plugins import HeatMap

#Calculate distance on latitude and longitude
from math import radians, cos, sin, asin, sqrt

#Library to find correlation in categorical data
from pandas import factorize

# Modeling
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import warnings
from sklearn.utils import resample
warnings.filterwarnings('ignore')


# **Analysis Segementation**

In this part we want explain our customers segementation, after we apply best method for segementation with Kmeans with 6 customers segementation based on Sillhoutte Score.

#### **Set Our Dataset, Preprocess, Scaled, Modeling Kmeans**

In [2]:
# Set Our DataSet

# Change the product name to english
products = pd.read_csv("olist_products_dataset.csv")
prod_cat_name_tr = pd.read_csv("product_category_name_translation.csv")
products = products.merge(prod_cat_name_tr, on='product_category_name', how='left')
products.drop ('product_category_name', axis=1, inplace=True)


#load all the tables
o_items = pd.read_csv("olist_order_items_dataset.csv")
orders = pd.read_csv("olist_orders_dataset.csv")
o_reviews = pd.read_csv("olist_order_reviews_dataset.csv")
geoloc = pd.read_csv("olist_geolocation_dataset.csv")
sellers = pd.read_csv("olist_sellers_dataset.csv")
o_payments = pd.read_csv("olist_order_payments_dataset.csv")
customers = pd.read_csv("olist_customers_dataset.csv")

# Join the needed table
df = orders.merge(o_items, on='order_id', how='left')
df = df.merge(o_payments, on='order_id', how='outer', validate='m:m')
df = df.merge(products, on='product_id', how='outer')
df = df.merge(customers, on='customer_id', how='outer')
df = df.merge(sellers, on='seller_id', how='outer')

In [3]:
# remove columns we don't need
df = df.drop(axis=1,
columns=['product_name_lenght',
'product_description_lenght',
'product_photos_qty',
'product_weight_g',
'product_length_cm',
'product_height_cm',
'product_width_cm',
'order_delivered_carrier_date',
'order_delivered_customer_date',
'order_estimated_delivery_date'
,'shipping_limit_date','order_approved_at'])

In [5]:
# Change to Datetime
df['order_purchase_date']=pd.to_datetime(df['order_purchase_timestamp'])

df['order_purchase_date'] = pd.to_datetime(df['order_purchase_timestamp'])
df['order_purchase_date'] = df['order_purchase_date'].dt.strftime('%Y-%m-%d-%H')
df['order_purchase_date'] = pd.to_datetime(df['order_purchase_date'])

In [6]:
df['year'] = df['order_purchase_date'].dt.year
df['month'] = df['order_purchase_date'].dt.strftime('%b')
df['day'] = df['order_purchase_date'].dt.day

In [7]:
# Preprocessing

df = df[(df['payment_type'] != 'not_defined') | (df['seller_city'] != '04482255') | (df['seller_city'] != 'vendas@creditparts.com.br') |
        (df['seller_city'] != 'rio de janeiro / rio de janeiro') | (df['seller_city'] != 'sao paulo / sao paulo') | (df['seller_city'] != 'rio de janeiro \\rio de janeiro') 
        | (df['seller_city'] != 'ribeirao preto / sao paulo' ) | (df['seller_city'] != 'sp') | (df['seller_city'] != 'carapicuiba / sao paulo') | (df['seller_city'] != 'mogi das cruzes / sp')
        | (df['seller_city'] != 'sp / sp') | (df['seller_city'] != 'auriflama/sp') | (df['seller_city'] != 'pinhais/pr') | (df['seller_city'] != 'cariacica / es') | (df['seller_city'] != 'jacarei / sao paulo')
        | (df['seller_city'] != 'sao sebastiao da grama/sp') | (df['seller_city'] != 'maua/sao paulo') | (df['seller_city'] != ' ') | (df['seller_city'] != 'lages - sc') 
        | (df['year'] != 2016) | (df['order_purchase_timestamp'] != '2018-09-03 09:06:57')]

In [8]:
df.dropna(axis=0,inplace=True,subset=['product_id', 'seller_id', 'payment_sequential','payment_type', 'payment_installments', 'payment_value'])

In [9]:
df['product_category_name_english'].fillna('Missing', inplace = True)

In [10]:
# Set RFM Analysis Data set
recency = df[['customer_unique_id', 'order_purchase_date']].copy()
recency = recency.groupby('customer_unique_id')['order_purchase_date'].max().reset_index()
recency.columns = ['customer_unique_id', 'last_purchase_timestamp']
# Recency, Frequency, Monetary
recency['purchase_int'] = (recency['last_purchase_timestamp'].max() - recency['last_purchase_timestamp']).dt.days
recency.drop(columns='last_purchase_timestamp', inplace=True)
frequency =pd.DataFrame(df.groupby('customer_unique_id')['order_id'].count().reset_index())
monetary =pd.DataFrame(df.groupby('customer_unique_id')['payment_value'].sum().reset_index())
# Merge
overall = recency.merge(frequency, on='customer_unique_id')
overall = overall.merge(monetary, on='customer_unique_id')

# Rename columns for better intepretation
overall.rename(columns={ 'purchase_int':'recency', 'num_transaction':'frequency','payment_value':'monetary','order_id':'frequency'}, inplace=True)

overall



,customer_unique_id,recency,frequency,monetary
0,0000366f3b9a7992bf8c76cfdf3221e2,115,1,141.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,118,1,27.19
2,0000f46a3911fa3c0805444483337064,541,1,86.22
3,0000f6ccb0745a6a4b88665a16c9f078,325,1,43.62
4,0004aac84e0df4da2b147fca70cf8255,292,1,196.89
...,...,...,...,...
95414,fffcf5a5ff07b0908bd4e2dbc735a684,451,2,4134.84
95415,fffea47cd6d3cc0a88bd621562a9d061,266,1,84.58
95416,ffff371b4d645b6ecea244b27531430a,572,1,112.46
95417,ffff5962728ec6157033ef9805bacc48,123,1,133.69


In [52]:
scaled_features = overall[['customer_unique_id','recency','frequency','monetary']].copy()

col_names = ['monetary', 'recency','frequency']
features = scaled_features[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)

scaled_features[col_names] = features
scaled_features

,customer_unique_id,recency,frequency,monetary
0,0000366f3b9a7992bf8c76cfdf3221e2,-0.833414,-0.282446,-0.110526
1,0000b849f77a49e4a4ce2b2a4ca5be3f,-0.813826,-0.282446,-0.289269
2,0000f46a3911fa3c0805444483337064,1.948149,-0.282446,-0.197287
3,0000f6ccb0745a6a4b88665a16c9f078,0.537779,-0.282446,-0.263667
4,0004aac84e0df4da2b147fca70cf8255,0.322306,-0.282446,-0.024840
...,...,...,...,...
95414,fffcf5a5ff07b0908bd4e2dbc735a684,1.360495,0.932534,6.111335
95415,fffea47cd6d3cc0a88bd621562a9d061,0.152539,-0.282446,-0.199843
95416,ffff371b4d645b6ecea244b27531430a,2.150564,-0.282446,-0.156400
95417,ffff5962728ec6157033ef9805bacc48,-0.781178,-0.282446,-0.123319


In [53]:
# Segmentation Based on RFM with KMeans
kmeans = KMeans(n_clusters=6,random_state= 0)
kmeans.fit(scaled_features[['monetary', 'recency','frequency']])

KMeans(n_clusters=6, random_state=0)

In [54]:
scaled_features['k_means_clust'] = kmeans.labels_
scaled_features


,customer_unique_id,recency,frequency,monetary,k_means_clust
0,0000366f3b9a7992bf8c76cfdf3221e2,-0.833414,-0.282446,-0.110526,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,-0.813826,-0.282446,-0.289269,1
2,0000f46a3911fa3c0805444483337064,1.948149,-0.282446,-0.197287,0
3,0000f6ccb0745a6a4b88665a16c9f078,0.537779,-0.282446,-0.263667,0
4,0004aac84e0df4da2b147fca70cf8255,0.322306,-0.282446,-0.024840,0
...,...,...,...,...,...
95414,fffcf5a5ff07b0908bd4e2dbc735a684,1.360495,0.932534,6.111335,4
95415,fffea47cd6d3cc0a88bd621562a9d061,0.152539,-0.282446,-0.199843,1
95416,ffff371b4d645b6ecea244b27531430a,2.150564,-0.282446,-0.156400,0
95417,ffff5962728ec6157033ef9805bacc48,-0.781178,-0.282446,-0.123319,1


In [55]:
scaled_features['k_means_clust'] = kmeans.labels_
scaled_features


,customer_unique_id,recency,frequency,monetary,k_means_clust
0,0000366f3b9a7992bf8c76cfdf3221e2,-0.833414,-0.282446,-0.110526,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,-0.813826,-0.282446,-0.289269,1
2,0000f46a3911fa3c0805444483337064,1.948149,-0.282446,-0.197287,0
3,0000f6ccb0745a6a4b88665a16c9f078,0.537779,-0.282446,-0.263667,0
4,0004aac84e0df4da2b147fca70cf8255,0.322306,-0.282446,-0.024840,0
...,...,...,...,...,...
95414,fffcf5a5ff07b0908bd4e2dbc735a684,1.360495,0.932534,6.111335,4
95415,fffea47cd6d3cc0a88bd621562a9d061,0.152539,-0.282446,-0.199843,1
95416,ffff371b4d645b6ecea244b27531430a,2.150564,-0.282446,-0.156400,0
95417,ffff5962728ec6157033ef9805bacc48,-0.781178,-0.282446,-0.123319,1


In [56]:
scaled_features['k_means_clust'].value_counts()

1    47860
0    35761
3    10821
4      950
2       26
5        1
Name: k_means_clust, dtype: int64

In [57]:
#assign customer segmentation based on overall score
ksegment_lst = scaled_features['k_means_clust']
ksegment_cluster = []

for i in ksegment_lst:
    if i == 1:
        cluster = 'passive'
        ksegment_cluster.append(cluster)
    elif i == 0:
        cluster = 'regular'
        ksegment_cluster.append(cluster)
    elif i == 3:
        cluster = 'occation'
        ksegment_cluster.append(cluster)
    elif i == 4:
        cluster = 'valuable'
        ksegment_cluster.append(cluster)
    elif i == 2:
        cluster = 'loyal'
        ksegment_cluster.append(cluster)
    else:
        cluster = 'best'
        ksegment_cluster.append(cluster)
        

In [58]:
scaled_features['k_means_segment'] = ksegment_cluster

## **Analysis Customers Based on Segementation**

In [126]:
# Merging Kmeans Segement for location & RFM before scaled

segment = scaled_features[['customer_unique_id','k_means_segment']].merge(df[['customer_unique_id','payment_type','customer_city','customer_state','year','month','product_category_name_english']], how='outer', on='customer_unique_id')
segment = segment.merge(overall, how='outer', on='customer_unique_id')
segment.head(5)


,customer_unique_id,k_means_segment,payment_type,customer_city,customer_state,year,month,product_category_name_english,recency,frequency,monetary
0,0000366f3b9a7992bf8c76cfdf3221e2,passive,credit_card,cajamar,SP,2018,May,bed_bath_table,115,1,141.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,passive,credit_card,osasco,SP,2018,May,health_beauty,118,1,27.19
2,0000f46a3911fa3c0805444483337064,regular,credit_card,sao jose,SC,2017,Mar,stationery,541,1,86.22
3,0000f6ccb0745a6a4b88665a16c9f078,regular,credit_card,belem,PA,2017,Oct,telephony,325,1,43.62
4,0004aac84e0df4da2b147fca70cf8255,regular,credit_card,sorocaba,SP,2017,Nov,telephony,292,1,196.89


### **Passive Customers**

In [127]:
# Numerikal Features Analysis

passive=segment[segment['k_means_segment'] == 'passive']
passive.describe()

,year,recency,frequency,monetary
count,47860.000000,47860.000000,47860.0,47860.000000
mean,2017.933326,135.400293,1.0,148.429274
std,0.249459,74.605649,0.0,181.563719
min,2017.000000,4.000000,1.0,10.890000
25%,2018.000000,70.000000,1.0,58.620000
50%,2018.000000,137.000000,1.0,100.340000
75%,2018.000000,198.000000,1.0,167.860000
max,2018.000000,269.000000,1.0,2512.530000


In [151]:
passive['monetary'].describe()[3]

10.89

**Passive Characteristic Based On Numerikal Features**

1.  Average Recency: 135 Days
    , Passive customers last purchase average was 135 days
2.  Average Frequency: 1
    , Passive customers only buy item once in our e-commerce
3.  Average Monetary: 148
    , Passive customers average spending in our e-commerce was R$148
4.  Most Transaction Passive customers was in 2018

In [158]:
# having the least spend on passive customers segment
passive[passive['monetary'] == passive['monetary'].describe()[3]]   

,customer_unique_id,k_means_segment,payment_type,customer_city,customer_state,year,month,product_category_name_english,recency,frequency,monetary,months
82509,b33336f46234b24a613ad9064d13106d,passive,credit_card,sao paulo,SP,2018,Jun,auto,73,1,10.89,5


**The least passive customers spend are R$ 10.89**
 
    - buy in the last 73 days
    - transaction using credit card
    - In City Sao Paulo
    - Buy Auto

In [159]:
# having the highest spend on passive customers segment
passive[passive['monetary'] == passive['monetary'].describe()[7]]   

,customer_unique_id,k_means_segment,payment_type,customer_city,customer_state,year,month,product_category_name_english,recency,frequency,monetary,months
1281,02ccceb879088a0253e1e46e0200bdb8,passive,debit_card,birigui,SP,2018,Aug,housewares,25,1,2512.53,7


**The highest passive customers spend are R$ 2512.53**

    - buy in the last 25 days
    - transaction using debit card
    - in City Birigui
    - Buy Housewares

In [136]:
ordered_months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun",
      "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

# sorting data accoring to ordered_months
passive['months']=passive['month'].apply(lambda x:ordered_months.index(x))
passive = passive.sort_values('months')

In [139]:
exclude_2017 = passive[passive['year'] != 2017]
agg= exclude_2017.groupby(['month','months','year'])['monetary'].sum().reset_index().sort_values(by='months')
agg

,month,months,year,monetary
3,Jan,0,2018,837334.87
2,Feb,1,2018,750909.82
6,Mar,2,2018,897547.07
0,Apr,3,2018,898722.03
7,May,4,2018,859637.26
5,Jun,5,2018,796150.30
4,Jul,6,2018,816752.40
1,Aug,7,2018,804050.47


**Exclude 2017 Year because most passive customers transaction in 2018**

In [140]:
px.line(agg, x="month" , y="monetary", color = 'year', line_group = 'year', title = 'Total Monetary by Passive Customers')

**Most Passive customers spend money in 2018 February and March ,There a decline gross sales in January and Mei we should investigate more**

In [115]:
# 10 State most of passive customers
top_10_customer_state = passive['customer_state'].value_counts().sort_values(ascending=False).head(10).index
df_top_10_customer_state = passive[passive['customer_state'].isin(top_10_customer_state)]

In [145]:
plt.figure(figsize=(8, 6))
df_price_cat = df_top_10_customer_state['customer_state'].value_counts()/len(passive['customer_city'])*100
pal = sns.color_palette('rocket_r', len(df_price_cat))
fig = px.bar(df_price_cat, x=df_price_cat.index, y= df_price_cat.values,color=df_price_cat.index)
fig.update_xaxes(title_text='State')
fig.update_yaxes(title_text='Percent Order')
fig.show()

<Figure size 576x432 with 0 Axes>

**As we know in Brazil Sao Paulo are the most densed State, it is normal 43% of our Passive customers are from Sao Paolo,its very densed in Sao Paulo**

In [102]:
# 10 City most of passive customers
top_10_customer_city = passive['customer_city'].value_counts().sort_values(ascending=False).head(10).index
df_top_10_customer_city = passive[passive['customer_city'].isin(top_10_customer_city)]

In [144]:
plt.figure(figsize=(8, 6))
df_price_cat = df_top_10_customer_city['customer_city'].value_counts()/len(passive['customer_city'])*100
pal = sns.color_palette('rocket_r', len(df_price_cat))
fig = px.bar(df_price_cat, x=df_price_cat.index, y= df_price_cat.values,color=df_price_cat.index)
fig.update_xaxes(title_text='City')
fig.update_yaxes(title_text='Percent Order')
fig.show()

<Figure size 576x432 with 0 Axes>

**Since Most our Customers from Sao Paulo State, ofcourse the capital city of this state will having the highest transaction**

In [143]:
df_price_cat = passive['payment_type'].value_counts()/len(passive['payment_type'])*100
fig = px.bar(df_price_cat, x=df_price_cat.index, y= df_price_cat.values,color=df_price_cat.index)
fig.show()

**Our most used payment method in passive Customers are Credit Card used 77%  and Boleto 19.5% for trasaction**

In [186]:
top_10_product = passive['product_category_name_english'].value_counts().sort_values(ascending=False).head(10).index
df_top_10_product = passive[passive['product_category_name_english'].isin(top_10_product)]

plt.figure(figsize=(8, 6))
df_price_cat = df_top_10_product['product_category_name_english'].value_counts()/len(passive['customer_city'])*100
pal = sns.color_palette('rocket_r', len(df_price_cat))
fig = px.bar(df_price_cat, x=df_price_cat.index, y= df_price_cat.values,color=df_price_cat.index)
fig.update_xaxes(title_text='Category')
fig.update_yaxes(title_text='Percent Order')
fig.show()

<Figure size 576x432 with 0 Axes>

**Most Product Passive Customers Buy are Bed Bath Table (8.2%), Health Beauty (10.17%), Sport Leisure(7.4%),etc**

#### **Conclusion Behavior Passive Customers**

    - Most of them in State: Sao Paolo, City: Sao Paolo
    - Most of them using Credit Cards(77%), a part of them using Boleto(19%)
    - Favorite Category: Bed Bath Table, Health Beauty, Sport Leisure, Watches Gift & Computers accesories
    - Most transaction in 2018
    - Their numbers decline

    Passive Characteristic Based On Numerikal Features
        1.  Average Recency: 135 Days
            Passive customers last purchase average was 135 days
        2.  Average Frequency: 1
            Passive customers only buy item once in our e-commerce
        3.  Average Monetary: 148
            Passive customers average spending in our e-commerce was R$148
        4.  Most Transaction Passive customers was in 2018
    - The least spend are R$ 10.89
    - The highest spend are R$ 2512.53
    

### **Regular Customers**


In [162]:
# Numerikal Features Analysis

regular=segment[segment['k_means_segment'] == 'regular']
regular.describe()

,year,recency,frequency,monetary
count,38374.000000,38374.000000,38374.000000,38374.000000
mean,2016.991218,405.720722,1.137541,168.806346
std,0.093301,94.961744,0.350274,209.564942
min,2016.000000,266.000000,1.000000,10.070000
25%,2017.000000,323.000000,1.000000,62.780000
50%,2017.000000,397.000000,1.000000,107.780000
75%,2017.000000,477.000000,1.000000,189.435000
max,2017.000000,728.000000,3.000000,4764.340000


**Regular Characteristic Based On Numerikal Features**

        1.  Average Recency: 405 Days
            Regular customers last purchase average was 135 days
        2.  Average Frequency: 1.13
            Regular customers only buy item once in our e-commerce
        3.  Average Monetary: 168
            Regular customers average spending in our e-commerce was R$168.8
        4.  Most Transaction Passive customers was in 2017

In [171]:
# having the least spend on passive customers segment
regular[regular['monetary'] == regular['monetary'].describe()[3]]   

,customer_unique_id,k_means_segment,payment_type,customer_city,customer_state,year,month,product_category_name_english,recency,frequency,monetary,months
86877,bd06ce0e06ad77a7f681f1a4960a3cc6,regular,credit_card,sao paulo,SP,2017,Sep,stationery,354,1,10.07,8


**The least passive customers spend are R$ 10.07**
 
    - buy in the last 354 days
    - transaction using credit card
    - In City Sao Paulo
    - Buy Stationery

In [172]:
# having the highest spend on passive customers segment
regular[regular['monetary'] == regular['monetary'].describe()[7]]   

,customer_unique_id,k_means_segment,payment_type,customer_city,customer_state,year,month,product_category_name_english,recency,frequency,monetary,months
109821,eebb5dda148d3893cdaf5b5ca3040ccb,regular,credit_card,maua,SP,2017,Apr,small_appliances,502,1,4764.34,3


**The least passive customers spend are R$ 4764.34**
 
    - buy in the last 503 days
    - transaction using credit card
    - In City Maua
    - Buy Small Appliances

In [164]:
ordered_months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun",
      "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

# sorting data accoring to ordered_months
regular['months']=regular['month'].apply(lambda x:ordered_months.index(x))
regular = regular.sort_values('months')

In [168]:
exclude_2016 = regular[regular['year'] != 2016]
agg= exclude_2016.groupby(['month','months','year'])['monetary'].sum().reset_index().sort_values(by='months')
agg

,month,months,year,monetary
4,Jan,0,2017,166966.52
3,Feb,1,2017,353389.89
7,Mar,2,2017,504117.40
0,Apr,3,2017,490157.24
8,May,4,2017,672061.20
6,Jun,5,2017,566051.17
5,Jul,6,2017,686289.38
1,Aug,7,2017,685004.23
11,Sep,8,2017,584573.21
10,Oct,9,2017,578230.20


**Reason Exclude 2016 the company data are so few**

In [169]:
px.line(agg, x="month" , y="monetary", color = 'year', line_group = 'year', title = 'Total Monetary by Regular Customers')

**The most highest spend in Regular Customers are in Nov, in our analysis Novermber was Black Friday time, after Black Friday over people din't so much spend money in December**

In [ ]:
# 10 State most of passive customers
top_10_customer_state = regular['customer_state'].value_counts().sort_values(ascending=False).head(10).index
df_top_10_customer_state = regular[regular['customer_state'].isin(top_10_customer_state)]

In [177]:
plt.figure(figsize=(8, 6))
df_price_cat = df_top_10_customer_state['customer_state'].value_counts()/len(regular['customer_city'])*100
pal = sns.color_palette('rocket_r', len(df_price_cat))
fig = px.bar(df_price_cat, x=df_price_cat.index, y= df_price_cat.values,color=df_price_cat.index,title='Regular Customers State')
fig.update_xaxes(title_text='State')
fig.update_yaxes(title_text='Percent Order')

fig.show()

<Figure size 576x432 with 0 Axes>

**As we know in Brazil Sao Paulo are the most densed State, it is normal 54% of our Regular customers are from Sao Paolo,its very densed in Sao Paulo**

In [179]:
# 10 City most of passive customers
top_10_customer_city = regular['customer_city'].value_counts().sort_values(ascending=False).head(10).index
df_top_10_customer_city = regular[regular['customer_city'].isin(top_10_customer_city)]

In [180]:
plt.figure(figsize=(8, 6))
df_price_cat = df_top_10_customer_city['customer_city'].value_counts()/len(regular['customer_city'])*100
pal = sns.color_palette('rocket_r', len(df_price_cat))
fig = px.bar(df_price_cat, x=df_price_cat.index, y= df_price_cat.values,color=df_price_cat.index,title='Regular Customers City')
fig.update_xaxes(title_text='City')
fig.update_yaxes(title_text='Percent Order')
fig.show()

<Figure size 576x432 with 0 Axes>

**Our Regular Customers As we can see not heavly dense in one city but scatter in many city of Sao Paulo State, but the 2nd Biggest transaction in Rio de Janeiro which din't belong in Sao Paulo State**

In [182]:
df_price_cat = regular['payment_type'].value_counts()/len(regular['payment_type'])*100
fig = px.bar(df_price_cat, x=df_price_cat.index, y= df_price_cat.values,color=df_price_cat.index)
fig.show()

**Credit Card and Boleto still are the most popular payment method**

In [185]:
top_10_product = regular['product_category_name_english'].value_counts().sort_values(ascending=False).head(10).index
df_top_10_product = regular[regular['product_category_name_english'].isin(top_10_product)]

plt.figure(figsize=(8, 6))
df_price_cat = df_top_10_product['product_category_name_english'].value_counts()/len(regular['product_category_name_english'])*100
fig = px.bar(df_price_cat, x=df_price_cat.index, y= df_price_cat.values,color=df_price_cat.index)
fig.update_xaxes(title_text='Category')
fig.update_yaxes(title_text='Percent Order')
fig.show()

<Figure size 576x432 with 0 Axes>

**Favorite buy category for regular customers are bed bath table(9.6%), sport leisure(8.1%), health beauty(7.7%), furniture decor(6.9%), computers accesories(5.8%), etc**

#### **Conclusion Behavior Regular Customers**

    - Most of them in State: Sao Paolo, Customers city are scatter around in Sao Paolo State but 2nd biggest are in Rio De janeiro that not in Sao Paolo State
    - Most of them using Credit Cards(75%), a part of them using Boleto(21%)
    - Favorite Category: Bed Bath Table, Health Beauty, Sport Leisure, Furniture Decor & Computers accesories
    - Most transaction in 2017
    - Their treds numbers decline

    Regular Characteristic Based On Numerikal Features

        1.  Average Recency: 405 Days
            Regular customers last purchase average was 135 days
        2.  Average Frequency: 1.13
            Regular customers only buy item once in our e-commerce
        3.  Average Monetary: 168
            Regular customers average spending in our e-commerce was R$168.8
        4.  Most Transaction Passive customers was in 2017
    - The least spend are R$ 10.07
    - The highest spend are R$ 4764.34
    